In [15]:
import pandas as pd
import numpy as np
import re
import time
import json
import glob
import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [16]:
# active leaning - é muito caro fazer anotações em novos exemplos
df = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df=df[df['y'].notnull()]

In [17]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

In [18]:
clean_date= df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x if len(x)==1 else x)
mapa_meses = {"jan":"Jan", "fev":"Feb", "mar":"Mar", "abr":"Apr", "mai":"May", "jun":"Jun","jul":"Jul", "ago":"Aug", "set":"Sep","out":"Oct","nov":"Nov","dez":"Dec"}
clean_date[1] = clean_date[1].map(mapa_meses)
clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

In [19]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)
df_limpo['views'] =views

<ipython-input-19-86d1434b4248>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)


In [20]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [21]:
features['tempo_desd_pub'] = (pd.to_datetime("2021-02-13") - df_limpo['date'])/np.timedelta64(1, "D")
features['views'] = df_limpo['views']
features['views_por_dias'] = features['views'] / features['tempo_desd_pub']
features = features.drop(['tempo_desd_pub'], axis =1)

In [22]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = df_limpo['date'] >= "2019-04-01"
Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((228, 2), (270, 2), (228,), (270,))

In [23]:
# da msi peso mais peso para palavras q aparecem muito em determinado exemplo mas nao aparece tanto pro dataset
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

In [25]:
title_vec= TfidfVectorizer(min_df=2)

In [26]:
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [27]:
from scipy.sparse import hstack, vstack
Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [28]:
mdl=RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [29]:
p=mdl.predict_proba(Xval_wtitle)[:,1]

In [30]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [31]:
average_precision_score(yval, p)

0.18465551413200237

In [32]:
df_unlabeled = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df_unlabeled = df_unlabeled[df_unlabeled['y'].isnull()].dropna(how='all')

In [33]:
df_limpo_u = pd.DataFrame(index=df_unlabeled.index)
df_limpo_u['title'] = df_unlabeled['watch-title']

In [34]:
clean_date= df_unlabeled['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x if len(x)==1 else x)
mapa_meses = {"jan":"Jan", "fev":"Feb", "mar":"Mar", "abr":"Apr", "mai":"May", "jun":"Jun","jul":"Jul", "ago":"Aug", "set":"Sep","out":"Oct","nov":"Nov","dez":"Dec"}
clean_date[1] = clean_date[1].map(mapa_meses)
clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
df_limpo_u['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

In [35]:
df_limpo_u.head()

,title,date
501,Kaggle Mercari Price Suggestion Challenge (1 p...,2018-11-02
502,OpenAI Gym and Python for Q-learning - Reinfor...,2018-10-14
503,"Dashboarding with Notebooks, Day 1: What infor...",2018-12-17
504,How To Get US- American Company H1 Visa To Get...,2019-11-23
505,Platform Overview - Machine Learning,2019-05-21


In [36]:
views = df_unlabeled['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)
df_limpo_u['views'] =views

<ipython-input-36-6700095517b9>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  views = df_unlabeled['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)


In [37]:
features_u = pd.DataFrame(index=df_limpo_u.index)

In [38]:
features_u['tempo_desd_pub'] = (pd.to_datetime("2021-02-13") - df_limpo_u['date'])/np.timedelta64(1, "D")
features_u['views'] = df_limpo_u['views']
features_u['views_por_dias'] = features_u['views'] / features_u['tempo_desd_pub']
features_u = features_u.drop(['tempo_desd_pub'], axis =1)

In [44]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
title_u = df_limpo_u['title']
title_bow_u = title_vec.transform(title_u)

In [46]:
Xu_wtitle=hstack([features_u, title_bow_u])

In [47]:
pu = mdl.predict_proba(Xu_wtitle)[:, 1]

In [48]:
df_unlabeled['p'] = pu

In [49]:
mask_u =(df_unlabeled['p']>=0.26) &(df_unlabeled['p']<=1.)
mask_u.sum()

84

In [50]:
df_unlabeled[mask_u].shape

(84, 17)

In [52]:
dificeis = df_unlabeled[mask_u]

In [54]:
aleatorios = df_unlabeled[~mask_u].sample(31, random_state=0) #selecione o que não é dessa máscara

In [ ]:
pd.concat([dificeis, aleatorios]).to_csv("active_label")